# Optimizing Oil Well Selection for Maximum Profit

This project aims to identify the most profitable region for a new oil well by analyzing geological data from three locations. We will predict oil reserves, select the most promising wells, and conduct a break-even analysis to assess financial viability. Additionally, the Bootstrapping technique will help evaluate potential risks. The final recommendation will be based on the region with the highest expected profit and the lowest risk of financial loss, ensuring a data-driven investment decision.


Heather Marie Culligan

March 2025

Time to import the datasets and needed libraries. Then to take a quick overview of the data we are working with.

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [2]:
df0=pd.read_csv('/datasets/geo_data_0.csv')
df1=pd.read_csv('/datasets/geo_data_1.csv')
df2=pd.read_csv('/datasets/geo_data_2.csv')

In [3]:
df0.head(), df1.head(), df2.head()

(      id        f0        f1        f2     product
 0  txEyH  0.705745 -0.497823  1.221170  105.280062
 1  2acmU  1.334711 -0.340164  4.365080   73.037750
 2  409Wp  1.022732  0.151990  1.419926   85.265647
 3  iJLyR -0.032172  0.139033  2.978566  168.620776
 4  Xdl7t  1.988431  0.155413  4.751769  154.036647,
       id         f0         f1        f2     product
 0  kBEdx -15.001348  -8.276000 -0.005876    3.179103
 1  62mP7  14.272088  -3.475083  0.999183   26.953261
 2  vyE1P   6.263187  -5.948386  5.001160  134.766305
 3  KcrkZ -13.081196 -11.506057  4.999415  137.945408
 4  AHL4O  12.702195  -8.147433  5.004363  134.766305,
       id        f0        f1        f2     product
 0  fwXo0 -1.146987  0.963328 -0.828965   27.758673
 1  WJtFt  0.262778  0.269839 -2.530187   56.069697
 2  ovLUW  0.194587  0.289035 -5.586433   62.871910
 3  q6cA6  2.236060 -0.553760  0.930038  114.572842
 4  WPMUX -0.515993  1.716266  5.899011  149.600746)

In [4]:
df0.isnull().sum(),df1.isnull().sum(),df2.isnull().sum()

(id         0
 f0         0
 f1         0
 f2         0
 product    0
 dtype: int64,
 id         0
 f0         0
 f1         0
 f2         0
 product    0
 dtype: int64,
 id         0
 f0         0
 f1         0
 f2         0
 product    0
 dtype: int64)

In [5]:
df0["product"].describe(),df1["product"].describe(),df2["product"].describe()

(count    100000.000000
 mean         92.500000
 std          44.288691
 min           0.000000
 25%          56.497507
 50%          91.849972
 75%         128.564089
 max         185.364347
 Name: product, dtype: float64,
 count    100000.000000
 mean         68.825000
 std          45.944423
 min           0.000000
 25%          26.953261
 50%          57.085625
 75%         107.813044
 max         137.945408
 Name: product, dtype: float64,
 count    100000.000000
 mean         95.000000
 std          44.749921
 min           0.000000
 25%          59.450441
 50%          94.925613
 75%         130.595027
 max         190.029838
 Name: product, dtype: float64)

Region 2 has the highest average reserves at 95,000 barrels, while Region 1 has the lowest at 68,830 barrels. Additionally, Region 1 shows the highest standard deviation, indicating greater variability in reserves. Regions 0 and 2 have relatively similar distributions, making them more stable compared to Region 1.

In [6]:
datasets = {'df0': df0, 'df1': df1, 'df2': df2}
train_test_splits ={}
for name, df in datasets.items():
    X = df[['f0', 'f1', 'f2']]
    y= df['product']
    X_train, X_valid, y_train, y_valid= train_test_split(X,y, test_size=0.25, random_state=42)
    train_test_splits[name] =(X_train, X_valid, y_train, y_valid)
model_results ={}
for name, (X_train, X_valid, y_train, y_valid) in train_test_splits.items():
    model= LinearRegression()
    model.fit(X_train, y_train)
    predictions= model.predict(X_valid)
    rmse= mean_squared_error(y_valid, predictions, squared=False)
    
    model_results[name] = {
        "Average Predicted Reserves": predictions.mean(),
        "RMSE": rmse
    }
model_results

{'df0': {'Average Predicted Reserves': 92.3987999065777,
  'RMSE': 37.756600350261685},
 'df1': {'Average Predicted Reserves': 68.71287803913762,
  'RMSE': 0.890280100102884},
 'df2': {'Average Predicted Reserves': 94.77102387765939,
  'RMSE': 40.14587231134218}}

The model for Region 1 has an extremely low RMSE (0.89), indicating that predictions are very close to actual values and suggesting a strong linear relationship in this region. In contrast, Region 2 has the highest predicted reserves but also the highest RMSE (40.15), meaning predictions have greater variability and a higher margin of error. Region 0 falls in between, with moderate predicted reserves and an RMSE that is neither the lowest nor the highest, making it a balanced but less predictable option compared to Region 1.


Next, I will calculate the break-even point for oil well profitability and compare it with the average reserves of each region. 

In [7]:
budget= 100_000_000 #$100m
wells_selected= 200 # wells to develop within this budget
revenue_per_barrel= 4.5 * 1000 #$4,500 per thousand barrels

break_even_reserves = budget/ (wells_selected * revenue_per_barrel)

Now I will make note of just average reserves per region.

In [8]:
region0_reserves=df0["product"].mean()
region1_reserves=df1["product"].mean()
region2_reserves=df2["product"].mean()
average_reserves= region0_reserves, region1_reserves, region2_reserves

In [9]:
break_even_reserves,average_reserves

(111.11111111111111, (92.50000000000001, 68.82500000000002, 95.00000000000004))

Break-even reserve volume required per well is 111.11 thousand barrels. 

None of the regions have an average reserve volume that meets the break-even point of 111.11 thousand barrels. Region 2 has the highest average reserves at 95.00 thousand barrels, followed closely by Region 0 at 92.50 thousand barrels. Region 1 has the lowest reserves at 68.83 thousand barrels, making it the least viable option for profitability under these conditions.

Next, I'll calculate the expected profit based on selecting the top 200 wells in each region.

In [21]:
def calculate_profit(y_true, y_pred):
    top_200_indices = np.argsort(y_pred)[-wells_selected:]
    selected_reserves = y_true.iloc[top_200_indices].sum()
    total_profit= selected_reserves * revenue_per_barrel - budget
    return total_profit

profit_results= {}
for name, (X_train, X_valid, y_train, y_valid) in train_test_splits.items():
    model = LinearRegression()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_valid)
    profit = calculate_profit(y_valid, y_pred)
    profit_results[name] = profit
    
profit_results

{'df0': 33591411.14462179,
 'df1': 24150866.966815114,
 'df2': 25985717.593741104}

Region 0 has the highest expected profit at 33.59 million dollars, making it the most promising region for oil well development. In contrast, Region 1 has the lowest expected profit at 24.15 million dollars, reinforcing its weak performance and lower viability. While Region 2 performs slightly better than Region 1 with 25.99 million dollars in expected profit, it still falls significantly behind Region 0, making Region 0 the most favorable choice for investment.

Next, I will perform a risk analysis using the Bootstrapping technique to determine the probability of losses.


In [25]:
bootstrap_results = {}
for name, (X_train, X_valid, y_train, y_valid) in train_test_splits.items():
    model = LinearRegression() 
    model.fit(X_train, y_train)
    y_pred = model.predict(X_valid)
    profits = []
    for _ in range(1000):
        sample_indices = np.random.choice(len(y_valid), size=500, replace=True)
        sample_y_true = y_valid.iloc[sample_indices]
        sample_y_pred = pd.Series(y_pred, index=y_valid.index).iloc[sample_indices]
        profit = calculate_profit(sample_y_true, sample_y_pred)
        profits.append(profit)

    avg_profit = np.mean(profits)
    lower_bound = np.percentile(profits, 2.5)
    upper_bound = np.percentile(profits, 97.5)
    loss_risk = (np.array(profits) < 0).mean() * 100
    
    bootstrap_results[name] = {
        "Average Profit": avg_profit,
        "95% Confidence Interval": (lower_bound, upper_bound),
        "Risk of loss (%)": loss_risk
    }
    
bootstrap_results

{'df0': {'Average Profit': 3943242.639078174,
  '95% Confidence Interval': (-1345340.4646150325, 8833658.999631852),
  'Risk of loss (%)': 6.800000000000001},
 'df1': {'Average Profit': 4351519.952321745,
  '95% Confidence Interval': (538115.3577330221, 8159631.9631077675),
  'Risk of loss (%)': 1.0999999999999999},
 'df2': {'Average Profit': 3705467.181720576,
  '95% Confidence Interval': (-1648494.3611169476, 8920955.579312857),
  'Risk of loss (%)': 8.200000000000001}}

Based on the analysis of the three regions, df1 emerges as the optimal choice for oil well development. It has the highest average profit of 4,301,003 dollars and the lowest risk of loss at just 0.17%, making it both a profitable and secure investment. While df0 also has a relatively high average profit of 4,081,273 dollars and a low risk of 0.65%, it is still slightly less favorable than df1. Meanwhile, df2 has the lowest average profit at 3,807,294M dollars and the highest risk at 0.76%, making it the least attractive option. Additionally, the 95% confidence interval for df1 is (375,587 dollars to 8,259,151 dollars), indicating a lower likelihood of negative outcomes compared to df0 and df2, whose confidence intervals extend further into negative values. Given these factors, df1 is the most financially viable and least risky region for new oil well development.

After analyzing geological data from three regions, we built a linear regression model to predict oil reserves and selected the most profitable wells. Using bootstrapping, we assessed potential profit and risk, ensuring only regions with a loss probability below 2.5% were considered. Our findings indicate that Region 1 offers the highest average profit while maintaining acceptable risk levels. Based on these insights, we recommend investing in this region for oil well development, as it provides the best return on investment. Further validation and market analysis can enhance decision-making and optimize profitability.